In [1]:
from fgglib.fgg.fgg import FGG
from fgglib.fg.fragment import Fragment
from fgglib.fgg.production import *
from fgglib.fgg.fggsum_product import FGGsum_product
from fgglib.base.semiring import Real
from fgglib.fg.functions.discretedensity import DiscreteDensity
from fgglib.autotesting.testenvironment import *

In [2]:
from fgglib.fg.factorgraph import Factorgraph
from fgglib.fg.fragment import Fragment
from fgglib.fg.vertex import Vertex, FGVertex
from fgglib.fg.edge import Edge, FGEdge
from fgglib.fg.variabledomain import VariableDomain
from fgglib.base.semiring import *

def createEdge(content,label,targets): # add this as a classmethod alternatively
    e = Edge(content,label)
    for t in targets:
        e.add_target(t)
    return e

def createFGEdge(content,label,targets, semiring=None):
    e = FGEdge(content,label,semiring, None)
    for t in targets:
        e.add_target(t)
    return e
    
def createFragment(vertexSet, edgeSet, external, semiring=None):
    frag = Fragment(semiring)
    for v in vertexSet:
        frag.add_vertex(v)
    for e in edgeSet:
        frag.add_edge(e)
    for ext in external:
        frag.add_external(ext)
    return frag

def createFGGraph(vertexSet, edgeSet, semiring):
    fg = Factorgraph(semiring)
    for v in vertexSet:
        fg.add_vertex(v)
    for e in edgeSet:
        fg.add_edge(e)
    return fg

def buildGraph(V, E, semiring): # prohibits use of label multiple times
    vertexDict = {l: FGVertex(None,l,semiring,defaultDomain) for l in V}
    vertexSet = {v for l,v in vertexDict.items()}
    edgeSet = set()
    for l,s in E.items():
        vs = [vertexDict[i] for i in s]
        edgeSet.add(createFGEdge(None,l,vs,semiring))
    return createFGGraph(vertexSet, edgeSet, semiring)
    
def buildFragment(V, E, ext, semiring=None): # prohibits use of vertex label multiple times
    vertexDict = {l: FGVertex(None,l,semiring,defaultDomain) for l in V}
    vertexSet = {v for i,v in vertexDict.items()}
    edgeSet = set()
    for tup in E:
        l = tup[0]
        s = tup[1]
        vs = [vertexDict[i] for i in s]
        edgeSet.add(createFGEdge(None,l,vs,semiring))
    external = {vertexDict[e] for e in ext}
    return createFragment(vertexSet, edgeSet, external, semiring)

defaultDomain = VariableDomain(False)
defaultDomain.set_content({0.25,0.5,0.75})
singularDomain = VariableDomain(False)
singularDomain.set_content({0.5})

In [3]:
finStatesFrag0 = buildFragment(
    ['A1', 'B2', 'A4'],
    [('X',['A1', 'B2', 'A4'])],
    {},
    Real
)

finStatesFrag1 = buildFragment(
    ['A1', 'B2'],
    [('Y',['A1', 'B2'])],
    {},
    Real
)

finStatesFrag2 = buildFragment(
    ['A1', 'B2', 'A4'],
    [('f',['A1', 'A4']), ('Y', ['B2', 'A4'])],
    {'A1', 'B2', 'A4'},
    Real
)

finStatesFrag3 = buildFragment(
    ['A1', 'B2'],
    [('g',['A1', 'B2'])],
    {'A1', 'B2'},
    Real
)

finStatesFGG = FGG(
    {finStatesFrag0, finStatesFrag1, finStatesFrag2, finStatesFrag3},
    {'S','X','Y'},
    'S',
    {Production('S', finStatesFrag0),
     Production('S', finStatesFrag1),
     Production('X', finStatesFrag2),
     Production('Y', finStatesFrag3)
    }
)

In [4]:
def test_inference_finite_states_example1():
    fggsp = FGGsum_product(finStatesFGG)
    
    finStatesDomain = VariableDomain(False)
    finStatesDomain.set_content({0, 1})
    finStatesFunction = DiscreteDensity([[0.1, 0.5],[0.3, 0.2]])
    
    for p in finStatesFGG.P:
        for v in p.body.V:
            v.domain = finStatesDomain
        for e in p.body.E:
            if e.label not in finStatesFGG.N:
                e.function = finStatesFunction
            
    
    
    fggsp.inference_finite_states()
    assert True

In [5]:
test_inference_finite_states_example1()

V-1643849817229052830|E133146708735736
{'S': Vertex: B_S in <class 'fgglib.base.semiring.Real'>,
 'X': Vertex: B_X in <class 'fgglib.base.semiring.Real'>,
 'Y': Vertex: B_Y in <class 'fgglib.base.semiring.Real'>}
{Production(head='S', body=V-8344747900437755062|E-3323496892668024800|X133146708735736): Vertex: B_π0 in <class 'fgglib.base.semiring.Real'>,
 Production(head='S', body=V-6979262830554793220|E-8852122404980006983|X133146708735736): Vertex: B_π2 in <class 'fgglib.base.semiring.Real'>,
 Production(head='X', body=V-6979262830554793220|E-4563026576679060480|X-6979262830554793220): Vertex: B_π1 in <class 'fgglib.base.semiring.Real'>,
 Production(head='Y', body=V-8344747900437755062|E7348197919872740307|X-8344747900437755062): Vertex: B_π3 in <class 'fgglib.base.semiring.Real'>}
{'X': {Vertex: A1 in <class 'fgglib.base.semiring.Real'>: Vertex: A1_X in <class 'fgglib.base.semiring.Real'>, Vertex: B2 in <class 'fgglib.base.semiring.Real'>: Vertex: B2_X in <class 'fgglib.base.semiring

KeyError: Vertex: A1 in <class 'fgglib.base.semiring.Real'>